In [37]:
import pandas as pd
import numpy as np
import sys
import datetime as dt
import time

# number of chunks in an hour
# e.g. 3 chunks would run 20-min shifts
chunks = 1

########################
# MISC FUNCTIONS
########################
def readTime(ti):
    return dt.datetime.strptime(ti, "%H:%M").time()

def rereadTime(ti):
    reread = str(ti)
    if len(reread) == 5: read = dt.datetime.strptime(reread, "%H:%M")
    else:                read = dt.datetime.strptime(reread, "%H:%M:%S")
    return read

def incrementTime(ti):
    return (rereadTime(ti) + dt.timedelta(hours=1/chunks)).time()

def dfFunction(df):
    DF = df.set_index(['time','car'])
    DF = DF.T.stack().T
    DF = DF.iloc[6*chunks:,:].append(DF.iloc[0:6*chunks,:])
    return DF

######################
# FOR VISUALISATION
######################
def color(val):
    color = 'green' if val > 0 else 'red'
    return 'color: %s' % color

def background(val):
    color = '#75fa7e' if val > 0 else '#fab9b9'
    return 'background-color: %s' % color

def markEvents(val):
    if val == 'idle': color = '#adfc83'
    elif val == 'charge': color = '#75fa7e'
    elif val == 'drive': color = '#fab9b9'
    elif val == 'RC': color = 'red'
    else: color = None
    return 'background-color: %s' % color

def styleDF(df):
    DF = df.style.\
        applymap(color, subset=['charge_rate']).\
        applymap(background, subset = ['charge_rate']).\
        applymap(markEvents, subset = ['event'])
    return DF

### Battery Functions

In [38]:
###############################
# DECREASE BATT DURING SHIFT
###############################
def decreaseBatt(carDataDF, shiftsByCar, time, simulationDF):
    for car in range(len(carDataDF)):
        # READ DATA FOR EVERY ROW IN CarDataDF
        batt = carDataDF.loc[car, 'battPerc']
        isC = carDataDF.loc[car, 'isCharge']
        battSize = carDataDF.loc[car, 'battSize']
        # CALCULATE RATE OF BATT DECREASE
        kwphr = mph/mpkw

        for b in range(0,len(shiftsByCar[str(car)])):
            startS = readTime(shiftsByCar[str(car)].loc[b, 'startShift'])
            endS = readTime(shiftsByCar[str(car)].loc[b, 'endShift'])

            # IF SHIFT DOESN'T RUN OVER MIDNIGHT
            if startS < endS:
                # DECREASE BATT DURING SHIFT
                if time >= startS and time < endS:
                    batt = carDataDF.loc[car,'battPerc']
                    simulationDF = simulationDF.append({
                        'time': time,
                        'car': car,
                        'charge_rate': 0,
                        'batt': round(batt, 2),
                        'event': 'drive' if batt-kwphr/chunks>0 else 'RC'
                    }, ignore_index=True)
                    batt -= kwphr/chunks

            # IF SHIFT RUNS OVER MIDNIGHT
            else:
                # SELECT NON-SHIFT TIME
                saveVal = startS
                startS = endS
                endS = saveVal

                # DECREASE BATT IF NOT DURING NON-SHIFT
                if time >= startS and time < endS: continue
                else:
                    batt = carDataDF.loc[car,'battPerc']
                    simulationDF = simulationDF.append({
                        'time': time,
                        'car': car,
                        'charge_rate': 0,
                        'batt': round(batt, 2),
                        'event': 'drive' if batt-kwphr/chunks>0 else 'RC'
                    }, ignore_index=True)
                    batt -= kwphr/chunks

        # RAPID CHARGE OUTSIDE CHARGE CENTRE IF VEHICLE HAS NO BATTERY
        if batt <= 0: batt = 27

        # ASSIGN BATTERY
        carDataDF.loc[car,'battPerc'] = batt

    return carDataDF, time, simulationDF

###################################
# CHARGE VEHICLE FOR ONE HOUR
###################################
def charge(carDataDF, carNum, chargeRate, simulationDF, time):
    batt = carDataDF.loc[carNum,'battPerc']
    battSize = carDataDF.loc[carNum,'battSize']
    simulationDF = simulationDF.append({
        'time': time,
        'car': carNum,
        'charge_rate': round(chargeRate, 2),
        'batt': round(batt, 2),
        'event': 'charge' if chargeRate > 0 else 'idle'
    }, ignore_index=True)
    
    # INCREASE BATT PERCENTAGE ACCORDING TO CHARGE RATE
    batt += chargeRate/chunks
    batt = battSize if batt >= battSize else batt
    carDataDF.loc[carNum, 'battPerc'] = batt
    
    return carDataDF, simulationDF

### Core Functions

In [39]:
#################################################################
# WHEN SHIFT STARTS: isCharge = 0 AND REMOVE FROM CHARGE CENTRE
# WHEN SHIFT ENDS: isCharge = 1 AND ENTER CHARGE CENTRE
#################################################################
def inOutCentre(carDataDF, shiftsByCar, time, chargeCen, simulationDF):
    for car in range(0, len(carDataDF)):
        for shifts in range(0, len(shiftsByCar[str(car)])):
            # READ DATA FOR EVERY ROW IN CarDataDF
            startS = readTime(shiftsByCar[str(car)].loc[shifts, 'startShift'])
            endS = readTime(shiftsByCar[str(car)].loc[shifts, 'endShift'])

            if time == startS:                      # exiting centre
                carDataDF.loc[car,'isCharge'] = 0
                chargeCen.remove(car)
            if time == endS:                        # entering centre
                carDataDF.loc[car,'isCharge'] = 1
                chargeCen.append(car)

    # SELECT IDLE VEHICLES
    chargeDF = carDataDF.loc[carDataDF['isCharge'] == 1]
    idleDF = chargeDF.loc[chargeDF['battPerc'] == 30]
    if len(idleDF) >= 1:
        for cars in range(len(idleDF)):
            num = idleDF.index[cars]
            simulationDF = simulationDF.append({
                'time': time,
                'car': num,
                'charge_rate': 0,
                'batt': round(carDataDF.loc[num,'battPerc'], 2),
                'event': 'idle'
            }, ignore_index=True)

    return carDataDF, time, chargeCen, simulationDF

#################################
# INCREASE BATT DURING CHARGE
#################################
def dumbCharge(carDataDF, chargeCen, chargeCapacity, maxRate, simulationDF):
    # IF THERE ARE CARS IN THE CHARGE CENTRE
    if len(chargeCen) >= 1:
        # SELECT CARS IN CENTRE WITH BATT < 100
        chargeDF = carDataDF.loc[carDataDF['isCharge'] == 1]
        chargeDF = chargeDF.loc[chargeDF['battPerc'] < 30]

        if len(chargeDF) >= 1:
            # CALCULATE CHARGE RATE
            chargeRate = chargeCapacity/len(chargeDF)
            if chargeRate > maxRate: chargeRate = maxRate

            # CHARGE SELECTED CARS IN CENTRE
            for cars in range(len(chargeDF)):
                num = chargeDF.index[cars]
                carDataDF, simulationDF = charge(carDataDF, num, chargeRate, simulationDF, time)
                
    return carDataDF, simulationDF

######################################
# INCREASE BATT DURING CHARGE (LEAVETIME)
######################################
def smartCharge_leavetime(carDataDF, chargeCen, shiftsByCar, time, chargeCapacity, maxRate, simulationDF):
    # IF THERE ARE CARS IN THE CHARGE CENTRE
    if len(chargeCen) >= 1:
        listRows = []
        # FIND THE TIMES WHEN CARS LEAVE THE CHARGE CENTRE
        for cars in range(0, len(chargeCen)):
            f = chargeCen[cars]
            leaveTime = readTime("23:59")
            for g in range(0, len(shiftsByCar[str(f)])):
                startTime = readTime(shiftsByCar[str(f)].loc[g, 'startShift'])
                if startTime > time and startTime < leaveTime:
                    leaveTime = startTime

            if leaveTime == readTime("23:59"):
                leaveTime = shiftsByCar[str(f)].loc[0,'startShift']

            hrsLeft = abs(rereadTime(leaveTime) - rereadTime(time))
            listRows.append([f, hrsLeft])

        leaveTimes = pd.DataFrame.from_records(listRows, columns=['car','hrsLeft'])
        leaveTimes = leaveTimes.sort_values(by=['hrsLeft'])
        leaveTimes = leaveTimes.reset_index(drop=True)

        for h in range(0, len(leaveTimes)):
            car = leaveTimes.loc[h, 'car']
            batt = carDataDF.loc[car, 'battPerc']
            batt_size = carDataDF.loc[car, 'battSize']

            if batt < batt_size:
                energyLeft = chargeCapacity - maxRate
                if energyLeft >= 0:
                    chargeRate = maxRate
                    carDataDF, simulationDF = charge(carDataDF, car, chargeRate, simulationDF, time)
                    chargeCapacity -= chargeRate

                elif energyLeft < 0 and energyLeft > -maxRate:
                    chargeRate = chargeCapacity
                    carDataDF, simulationDF = charge(carDataDF, car, chargeRate, simulationDF, time)
                    chargeCapacity -= chargeRate

                else:
                    # if vehicle is plugged in but not allocated any charge
                    chargeRate = 0
                    carDataDF, simulationDF = charge(carDataDF, car, chargeRate, simulationDF, time)

    return carDataDF, simulationDF

######################################
# INCREASE BATT DURING CHARGE (BATT)
######################################
def smartCharge_batt(carDataDF, chargeCen, shiftsByCar, time, chargeCapacity, maxRate, simulationDF):
    # IF THERE ARE CARS IN THE CHARGE CENTRE
    if len(chargeCen) >= 1:
        listRows = []
        # FIND THE TIMES WHEN CARS LEAVE THE CHARGE CENTRE
        for cars in range(0, len(chargeCen)):
            f = chargeCen[cars]
            leaveTime = readTime("23:59")
            for g in range(0, len(shiftsByCar[str(f)])):
                startTime = readTime(shiftsByCar[str(f)].loc[g, 'startShift'])
                if startTime > time and startTime < leaveTime:
                    leaveTime = startTime

            if leaveTime == readTime("23:59"):
                leaveTime = shiftsByCar[str(f)].loc[0,'startShift']

            battLeft = abs(carDataDF.loc[f,'battSize']-carDataDF.loc[f,'battPerc'])
            listRows.append([f, battLeft])

        leaveTimes = pd.DataFrame.from_records(listRows, columns=['car','battLeft'])
        leaveTimes = leaveTimes.sort_values(by=['battLeft'], ascending=True)
        leaveTimes = leaveTimes.reset_index(drop=True)

        for h in range(0, len(leaveTimes)):
            car = leaveTimes.loc[h, 'car']
            batt = carDataDF.loc[car, 'battPerc']
            batt_size = carDataDF.loc[car, 'battSize']

            if batt < batt_size:
                energyLeft = chargeCapacity - maxRate
                if energyLeft >= 0:
                    chargeRate = maxRate
                    carDataDF, simulationDF = charge(carDataDF, car, chargeRate, simulationDF, time)
                    chargeCapacity -= chargeRate

                elif energyLeft < 0 and energyLeft > -maxRate:
                    chargeRate = chargeCapacity
                    carDataDF, simulationDF = charge(carDataDF, car, chargeRate, simulationDF, time)
                    chargeCapacity -= chargeRate

                else:
                    # if vehicle is plugged in but not allocated any charge
                    chargeRate = 0
                    carDataDF, simulationDF = charge(carDataDF, car, chargeRate, simulationDF, time)

    return carDataDF, simulationDF

############################################
# INCREASE BATT DURING CHARGE (SUPER SMART)
############################################
def superSmartCharge(carDataDF, chargeCen, shiftsByCar, time, chargeCapacity, maxRate, simulationDF):
    # IF THERE ARE CARS IN THE CHARGE CENTRE
    if len(chargeCen) >= 1:
        listRows = []
        # FIND THE TIMES WHEN CARS LEAVE THE CHARGE CENTRE
        for cars in range(0, len(chargeCen)):
            f = chargeCen[cars]
            leaveTime = readTime("23:59")
            for g in range(0, len(shiftsByCar[str(f)])):
                startTime = readTime(shiftsByCar[str(f)].loc[g, 'startShift'])
                if startTime > time and startTime < leaveTime:
                    leaveTime = startTime

            if leaveTime == readTime("23:59"):
                leaveTime = shiftsByCar[str(f)].loc[0,'startShift']

            hrsLeft = abs(rereadTime(leaveTime) - rereadTime(time))
            battLeft = abs(carDataDF.loc[f,'battSize']-carDataDF.loc[f,'battPerc'])
            listRows.append([f, battLeft/hrsLeft.total_seconds(), battLeft])

        leaveTimes = pd.DataFrame.from_records(listRows, columns=['car','priority','battLeft'])
        leaveTimes = leaveTimes.sort_values(by=['battLeft'], ascending=True)
        prioritySum = sum(leaveTimes.priority)                

        for h in range(0, len(leaveTimes)):
            car = leaveTimes.loc[h, 'car']
            batt = carDataDF.loc[car, 'battPerc']
            batt_size = carDataDF.loc[car, 'battSize']
            batt_left = leaveTimes.loc[h, 'battLeft']
            priority = leaveTimes.loc[h, 'priority']

            if batt < batt_size:
                chargeRate = (priority/prioritySum)*chargeCapacity
                
                # if charge rate exceeds 7
                if chargeRate > maxRate: chargeRate = maxRate
                # if charge rate exceeds charge needed
                if chargeRate > batt_left: chargeRate = batt_left
                    
                chargeCapacity -= chargeRate
                prioritySum -= priority
                carDataDF, simulationDF = charge(carDataDF, car, chargeRate, simulationDF, time)

    return carDataDF, simulationDF

### Start Simulation

In [40]:
######################## DEPO ############################
chargeCapacity = 12             # SET MAX AVAILABLE POWER IN CENTRE (kW/hr)
maxRate = 7                     # SET MAX CHARGE RATE PER CAR (kW/hr)
######################## DRIVING ############################
mpkw = 4                        # SET AVERAGE MILES PER kW THAT WILL DETERMINE RATE OF BATT DECREASE
mph = 16                        # SET AVERAGE MILES PER HR COVERED
runTime = 24                    # CHOOSE RUNTIME (HRS)
carData = [[30, 1, 30], [30, 1, 30], [30, 1, 30], [30, 1, 30]]
car_cols = ["battPerc","isCharge","battSize"]
sim_cols = ['time','car','charge_rate','batt','event']

######################## DEPO ############################
chargeCapacity = 18             # SET MAX AVAILABLE POWER IN CENTRE (kW/hr)
maxRate = 8                     # SET MAX CHARGE RATE PER CAR (kW/hr)
######################## DRIVING ############################
mpkw = 4                        # SET AVERAGE MILES PER kW THAT WILL DETERMINE RATE OF BATT DECREASE
mph = 16                        # SET AVERAGE MILES PER HR COVERED
runTime = 24                    # CHOOSE RUNTIME (HRS)
carData = [[30, 1, 30], [30, 1, 30], [30, 1, 30], [30, 1, 30],
           [30, 1, 30], [30, 1, 30], [30, 1, 30], [30, 1, 30]]
car_cols = ["battPerc","isCharge","battSize"]
sim_cols = ['time','car','charge_rate','batt','event']

filename = "8_cars_first_shift_same" # very interesting example
carShifts = [[["07:00", "14:00"], ["20:00", "22:00"]],
             [["07:00", "14:00"], ["17:00", "20:00"]], 
             [["07:00", "14:00"], ["20:00", "00:00"]], 
             [["07:00", "14:00"], ["18:00", "23:00"]],
             [["07:00", "14:00"], ["20:00", "22:00"]],
             [["07:00", "14:00"], ["17:00", "20:00"]], 
             [["07:00", "14:00"], ["20:00", "00:00"]], 
             [["07:00", "14:00"], ["18:00", "23:00"]]]

shiftsByCar = {}                                                # Set dictionary name as 'shiftsByCar'
for cars in range(0,len(carData)):                              # For every keys of the car:
    shiftsDF = pd.DataFrame(carShifts[cars], columns=["startShift","endShift"])
    shiftsDF = shiftsDF.sort_values(by=['startShift'])
    shiftsDF = shiftsDF.reset_index(drop=True)
    shiftsByCar['%s' % cars] = shiftsDF                             # The value = an empty list
    
    

##################
# DUMB CHARGING
##################
chargeCen = []
carDataDF = pd.DataFrame.from_records(carData, columns=car_cols)
for car in range(0, len(carDataDF)):
    if carDataDF.loc[car,'isCharge']: chargeCen.append(car)

time = readTime("06:00")        # CHOOSE START TIME
simulationDF = pd.DataFrame(columns=sim_cols)

for i in range(0, runTime*chunks):
    carDataDF, time, chargeCen, simulationDF = inOutCentre(carDataDF, shiftsByCar, time, chargeCen, simulationDF)
    carDataDF, time, simulationDF = decreaseBatt(carDataDF, shiftsByCar, time, simulationDF)
    carDataDF, simulationDF = dumbCharge(carDataDF, chargeCen, chargeCapacity, maxRate, simulationDF)
    time = incrementTime(time)
dumb_sim = dfFunction(simulationDF)
dumbDF = styleDF(dumb_sim)


###########################
# SMART CHARGING LEAVETIME
###########################
chargeCen = []
carDataDF = pd.DataFrame.from_records(carData, columns=car_cols)
for car in range(0, len(carDataDF)):
    if carDataDF.loc[car,'isCharge']: chargeCen.append(car)

time = readTime("06:00")        # CHOOSE START TIME
simulationDF = pd.DataFrame(columns=sim_cols)

for i in range(0, runTime*chunks):
    carDataDF, time, chargeCen, simulationDF = inOutCentre(carDataDF, shiftsByCar, time, chargeCen, simulationDF)
    carDataDF, time, simulationDF = decreaseBatt(carDataDF, shiftsByCar, time, simulationDF)
    carDataDF, simulationDF = smartCharge_leavetime(carDataDF, chargeCen, shiftsByCar, time, chargeCapacity, maxRate, simulationDF)
    time = incrementTime(time)
smartCharge_leavetime_sim = dfFunction(simulationDF)
smartCharge_leavetimeDF = styleDF(smartCharge_leavetime_sim)


###########################
# SMART CHARGING BATT
###########################
chargeCen = []
carDataDF = pd.DataFrame.from_records(carData, columns=car_cols)
for car in range(0, len(carDataDF)):
    if carDataDF.loc[car,'isCharge']: chargeCen.append(car)

time = readTime("06:00")        # CHOOSE START TIME
simulationDF = pd.DataFrame(columns=sim_cols)

for i in range(0, runTime*chunks):
    carDataDF, time, chargeCen, simulationDF = inOutCentre(carDataDF, shiftsByCar, time, chargeCen, simulationDF)
    carDataDF, time, simulationDF = decreaseBatt(carDataDF, shiftsByCar, time, simulationDF)
    carDataDF, simulationDF = smartCharge_batt(carDataDF, chargeCen, shiftsByCar, time, chargeCapacity, maxRate, simulationDF)
    time = incrementTime(time)
smartCharge_batt_sim = dfFunction(simulationDF)
smartCharge_battDF = styleDF(smartCharge_batt_sim)


###########################
# SUPER SMART CHARGING
###########################
chargeCen = []
carDataDF = pd.DataFrame.from_records(carData, columns=car_cols)
for car in range(0, len(carDataDF)):
    if carDataDF.loc[car,'isCharge']: chargeCen.append(car)

time = readTime("06:00")        # CHOOSE START TIME
simulationDF = pd.DataFrame(columns=sim_cols)

for i in range(0, runTime*chunks):
    carDataDF, time, chargeCen, simulationDF = inOutCentre(carDataDF, shiftsByCar, time, chargeCen, simulationDF)
    carDataDF, time, simulationDF = decreaseBatt(carDataDF, shiftsByCar, time, simulationDF)
    carDataDF, simulationDF = superSmartCharge(carDataDF, chargeCen, shiftsByCar, time, chargeCapacity, maxRate, simulationDF)
    time = incrementTime(time)
smart_sim = dfFunction(simulationDF)
smartDF = styleDF(smart_sim)


# ###############################################################
# # SAVE TO EXCEL (ONLY RUN WHEN ALL ALGORITHMS ARE UNCOMMENTED)
# # NOTE: CREATE A FOLDER CALLED 'TEST' FIRST
# ###############################################################
# # open writer
# writer = pd.ExcelWriter("test/" + filename + ".xlsx")
# # write files
# dumbDF.to_excel(
#     writer, sheet_name="dumb")
# smart_leavetimeDF.to_excel(
#     writer, sheet_name="smart_leavetime")
# smart_battDF.to_excel(
#     writer, sheet_name="smart_batt")
# smartDF.to_excel(
#     writer, sheet_name="superSmart")
# # close writer
# writer.save()

# smart_sim[['event','batt']].to_excel('output.xlsx')

In [41]:
smartDF